Biostat/Biomath M257 Homework 5

Due May 24 @ 11:59PM

Tomoki Okuno and 805851067

System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.10.2
Commit bd47eca2c8a (2024-03-01 10:14 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M1
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
Threads: 1 default, 0 interactive, 1 GC (on 4 virtual cores)
Environment:
  JULIA_NUM_THREADS = 


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()
# Pkg.add("MathOptInterface")
# Pkg.add("Mosek")

  Activating project at `~/Documents/07_UCLA/Class/257/02_Homework/hw5`


Status `~/Documents/07_UCLA/Class/257/02_Homework/hw5/Project.toml`
  [1e616198] COSMO v0.8.9
  [61c947e1] Clarabel v0.7.1
  [f65535da] Convex v0.15.4
  [a93c6f00] DataFrames v1.6.1
  [60bf3e95] GLPK v1.2.1
  [2e9cd046] Gurobi v1.2.3
  [87dc4568] HiGHS v1.9.0
  [b99e6be6] Hypatia v0.8.1
  [4076af6c] JuMP v1.22.1
  [67920dd8] KNITRO v0.14.2
  [b8f27783] MathOptInterface v1.29.0
  [6405355b] Mosek v10.1.4
  [1ec41992] MosekTools v0.15.1
  [2f354839] Pajarito v0.8.2
  [08abe8d2] PrettyTables v2.3.1
  [c946c3f1] SCS v2.0.0
  [3eaba693] StatsModels v0.7.3


In this exercise, we practice using disciplined convex programming (SDP in particular) to solve optimal design problems.

## Introduction to optimal design

Consider a linear model
\begin{aligned}
	y_i = \mathbf{x}_i^T \boldsymbol{\beta} + \epsilon_i, \quad i = 1,\ldots, n,
\end{aligned}
where $\epsilon_i$ are independent Gaussian noises with common variance $\sigma^2$. It is well known that the least squares estimate $\hat{\boldsymbol{\beta}}$ is unbiased and has covariance $\sigma^2 (\sum_{i=1}^n \mathbf{x}_i \mathbf{x}_i^T)^{-1}$. 

In **exact optimal design**, given total number of $n$ allowable experiments, we want to choose among a list of $m$ candidate design points $\{\mathbf{x}_1, \ldots, \mathbf{x}_m\}$ such that the covariance matrix is minimized in some sense. In mathematical terms, we want to find an integer vector $\mathbf{n} = (n_1, \ldots, n_m)$ such that $n_i \ge 0$, $\sum_{i=1}^m n_i = n$, and the matrix $\mathbf{V} = \left( \sum_{i=1}^m n_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1}$ is "small".

In **approximate optimal design**,  we want to find a probability vector $\mathbf{p} = (p_1, \ldots, p_m)$ such that $p_i \ge 0$, $\sum_{i=1}^m p_i = 1$, and the matrix $\mathbf{V} = \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1}$ is "small".

Commonly used optimal design criteria include:

- In **$D$-optimal design**, we minimize the determinant of $\mathbf{V}$
\begin{aligned}
	&\text{minimize}& \det \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.
\end{aligned}

- In **$E$-optimal design**, we minimize the spectral norm, i.e., the maximum eigenvalue of $\mathbf{V}$
\begin{aligned}
	&\text{minimize}& \lambda_{\text{max}} \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.	
\end{aligned}
Statistically we are minimizing the maximum variance of $\sum_{j=1}^p a_j \text{var}(\hat \beta_j)$ over all vectors $\mathbf{a}$ with unit norm.

- In **$A$-optimal design**, we minimize the trace of $\mathbf{V}$
\begin{aligned}
	&\text{minimize}& \text{tr} \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.
\end{aligned}
Statistically we are minimizing the total variance $\sum_{j=1}^p \text{var}(\hat \beta_j)$.

## Q1 (10 pts) 3x4 factorial design

A drug company asks you to help design a two factor clinical trial, in which treatment A has three levels (A1, A2, and A3) and treatment B has four levels (B1, B2, B3, and B4). Drug company also tells you that the treatment combination A3:B4 has undesirable side effects so we ignore this design point. 

Using dummy coding with A1 and B1 as the baseline levels, find the matrix $\mathbf{C}$ with each row a unique design point.

**Solution**

$\mathbf{C}$ has $6$ columns (intercept, A2, A3, B2, B3, B4) and $m = 3 \times 4 - 1 = 11$ rows when ignoring A3:B4. The row order follows the sequence seen in Q2:
$$
\begin{aligned}
\mathbf{C} = 
    \begin{bmatrix}
        \mathbf{x}_1^T\\
        \mathbf{x}_2^T\\
        \mathbf{x}_3^T\\
        \mathbf{x}_4^T\\
        \mathbf{x}_5^T\\
        \mathbf{x}_6^T\\
        \mathbf{x}_7^T\\
        \mathbf{x}_8^T\\
        \mathbf{x}_9^T\\
        \mathbf{x}_{10}^T\\
        \mathbf{x}_{11}^T
    \end{bmatrix}
    =
    \begin{bmatrix}
        1 & 0 & 0 & 0 & 0 & 0 \\
        1 & 1 & 0 & 0 & 0 & 0 \\
        1 & 0 & 1 & 0 & 0 & 0 \\
        1 & 0 & 0 & 1 & 0 & 0 \\
        1 & 1 & 0 & 1 & 0 & 0 \\
        1 & 0 & 1 & 1 & 0 & 0 \\
        1 & 0 & 0 & 0 & 1 & 0 \\
        1 & 1 & 0 & 0 & 1 & 0 \\
        1 & 0 & 1 & 0 & 1 & 0 \\
        1 & 0 & 0 & 0 & 0 & 1 \\
        1 & 1 & 0 & 0 & 0 & 1
    \end{bmatrix}.
\end{aligned}
$$
Suppose $\mathbf{W}$ is a diagonal matrix with positive entries of either $\mathbf{n}$ or $\mathbf{p}$, then we have
$$
\mathbf{V} = \left(\mathbf{C}^T\mathbf{W}\mathbf{C}\right)^{-1}
\Longleftrightarrow
\mathbf{V}^{-1} = \mathbf{C}^T\mathbf{W}\mathbf{C} \quad\text{(information)}
$$
with constraint $\mathbf{1}^T\mathbf{n} = n$ or $\mathbf{1}^T\mathbf{p} = 1$.

<!-- \begin{array}{c|ccccc}
    & \text{Intercept} & \text{A2} & \text{A3} & \text{B2} & \text{B3} & \text{B4} \\
    \hline
    \text{A1:B1} & 1 & 0 & 0 & 0 & 0 & 0 \\
    \text{A2:B1} & 1 & 1 & 0 & 0 & 0 & 0 \\
    \text{A3:B1} & 1 & 0 & 1 & 0 & 0 & 0 \\
    \text{A1:B2} & 1 & 0 & 0 & 1 & 0 & 0 \\
    \text{A2:B2} & 1 & 1 & 0 & 1 & 0 & 0 \\
    \text{A3:B2} & 1 & 0 & 1 & 1 & 0 & 0 \\
    \text{A1:B3} & 1 & 0 & 0 & 0 & 1 & 0 \\
    \text{A2:B3} & 1 & 1 & 0 & 0 & 1 & 0 \\
    \text{A3:B3} & 1 & 0 & 1 & 0 & 1 & 0 \\
    \text{A1:B4} & 1 & 0 & 0 & 0 & 0 & 1 \\
    \text{A2:B4} & 1 & 1 & 0 & 0 & 0 & 1
\end{array} -->

In [3]:
C = [1 0 0 0 0 0;
     1 1 0 0 0 0;
     1 0 1 0 0 0;
     1 0 0 1 0 0;
     1 1 0 1 0 0;
     1 0 1 1 0 0;
     1 0 0 0 1 0;
     1 1 0 0 1 0;
     1 0 1 0 1 0;
     1 0 0 0 0 1;
     1 1 0 0 0 1]

11×6 Matrix{Int64}:
 1  0  0  0  0  0
 1  1  0  0  0  0
 1  0  1  0  0  0
 1  0  0  1  0  0
 1  1  0  1  0  0
 1  0  1  1  0  0
 1  0  0  0  1  0
 1  1  0  0  1  0
 1  0  1  0  1  0
 1  0  0  0  0  1
 1  1  0  0  0  1

## Q2 (30 pts) Find approximate optimal designs

Using semidefinite programming (SDP) software to find the approximate D-, E-, and A-optimal designs for this clinical trial.

Hint: This is what I got (using Hypatia solver), which may or may not be correct.

```
Approximate Optimal Design
┌───────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│ design_pt │   D_opt │   E_opt │   A_opt │ D_opt_n │ E_opt_n │ A_opt_n │
│    String │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├───────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│      A1B1 │   0.082 │   0.271 │   0.200 │   8.000 │  27.000 │  20.000 │
│      A2B1 │   0.082 │   0.150 │   0.101 │   8.000 │  15.000 │  10.000 │
│      A3B1 │   0.097 │   0.118 │   0.104 │  10.000 │  12.000 │  10.000 │
│      A1B2 │   0.082 │   0.058 │   0.086 │   8.000 │   6.000 │   9.000 │
│      A2B2 │   0.082 │   0.037 │   0.051 │   8.000 │   4.000 │   5.000 │
│      A3B2 │   0.097 │   0.059 │   0.068 │  10.000 │   6.000 │   7.000 │
│      A1B3 │   0.082 │   0.058 │   0.086 │   8.000 │   6.000 │   9.000 │
│      A2B3 │   0.082 │   0.037 │   0.051 │   8.000 │   3.000 │   5.000 │
│      A3B3 │   0.097 │   0.059 │   0.068 │  10.000 │   6.000 │   7.000 │
│      A1B4 │   0.109 │   0.079 │   0.106 │  11.000 │   8.000 │  10.000 │
│      A2B4 │   0.109 │   0.075 │   0.080 │  11.000 │   7.000 │   8.000 │
│       Obj │   8.987 │  13.000 │  38.925 │   8.988 │  13.028 │  38.946 │
└───────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┘
```

**Solution**

**$D$-optimal design**:

Since $\det(\mathbf{V}) = [\det(\mathbf{V^{-1}})]^{-1} = [\det(\mathbf{C}^T\mathbf{W}\mathbf{C})]^{-1}$ and $-\log(x)$ is convex,
$$
\begin{aligned}
	&\text{minimize} \quad \det (\mathbf{V})\\
	\Longleftrightarrow\quad
	&\text{maximize} \quad \det(\mathbf{C}^T\mathbf{W}\mathbf{C})\\
	\Longleftrightarrow\quad
	&\text{maximize} \quad \det(\mathbf{C}^T\mathbf{W}\mathbf{C})\\
	\Longleftrightarrow\quad
	&\text{minimize} \quad -\text{logdet}\left(\mathbf{C}^T\mathbf{W}\mathbf{C}\right).
\end{aligned}
$$
So, we don't need to calculate the inverse. I use `minimize` (instead of `maximize`) to obtain the same (or a similar) objective function as Dr. Zhou did. Also, I use the Hypatia solver as well.

In [4]:
using LinearAlgebra
using Convex, Hypatia, MathOptInterface
using JuMP, Pajarito, Gurobi, Mosek, MosekTools

In [5]:
m = size(C, 1) # 11
p = Variable(m, Positive())
V⁻¹ = transpose(C) * Diagonal(p) * C
problem = minimize(-logdet(V⁻¹))
problem.constraints += p .>= 0
problem.constraints += sum(p) == 1
problem

const MOI = MathOptInterface
solver = Hypatia.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("verbose"), 0)
@time solve!(problem, solver)

# Check optimal value and optimal solution
D_opt_obj = round(problem.optval, digits = 3)
D_opt_p = round.(p.value, digits = 3)
println("Obj = ", D_opt_obj)

# store p and obj for later use
vcat(D_opt_p, [D_opt_obj])

 22.650425 seconds (44.61 M allocations: 3.019 GiB, 4.96% gc time, 99.83% compilation time: <1% of which was recompilation)
Obj = 8.987


12×1 Matrix{Float64}:
 0.082
 0.082
 0.097
 0.082
 0.082
 0.097
 0.082
 0.082
 0.097
 0.109
 0.109
 8.987

In [6]:
# check constraint satisfication
sum(p.value) ≈ 1.0

true

Multiplying $\mathbf p$ by 100 is not an effective way to convert $\mathbf{p}$ to $\mathbf{n}$ because it does not always result in $n=100$. Instead, use MISDP here as well as Q3.

In [7]:
# define a function to project a vector to the integer simplex
solver_misdp = 
    optimizer_with_attributes(
        Pajarito.Optimizer,
        MOI.Silent() => true,
        "oa_solver" => optimizer_with_attributes(
            Mosek.Optimizer,
            MOI.Silent() => true
        ),
        "conic_solver" => optimizer_with_attributes(
            Hypatia.Optimizer,
            MOI.Silent() => true
        )
    )

function proj_integer_simplex(
    p      :: Vector{Float64},
    ntotal :: Int;
    solver = solver_misdp
)
    n = Convex.Variable(length(p), IntVar)
    problem = minimize(norm(n - p))
    problem.constraints += sum(n) == ntotal
    solve!(problem, solver)
    round.(n.value, digits = 0) |> vec
end

proj_integer_simplex (generic function with 1 method)

In [8]:
D_opt_n = proj_integer_simplex(100vec(p.value), 100)

11-element Vector{Float64}:
  8.0
  8.0
 10.0
  8.0
  8.0
 10.0
  8.0
  8.0
 10.0
 11.0
 11.0

In [9]:
# check constraint satisfication
sum(D_opt_n) == 100

true

In [10]:
# calculate the corresponding obj
D_opt_obj_n = -logdet(transpose(C) * Diagonal(vec(D_opt_n)/100) * C)
D_opt_obj_n = round(D_opt_obj_n, digits = 3)

8.988

**$E$-optimal design**

It is important to note that the eigenvalue of the inverse of a matrix is the reciprocal of the eigenvalue of the original matrix.

Use the equivalent objective function to avoid taking inverse as shown below:
$$
\begin{aligned}
	&\text{minimize} \quad\lambda_{\text{max}}(\mathbf{V})\\
	\Longleftrightarrow\quad
	&\text{maximize} \quad \lambda_{\text{min}}\left(\mathbf{C}^T\mathbf{W}\mathbf{C}\right)\\
	\Longleftrightarrow\quad
	&\text{minimize} \quad 1/\lambda_{\text{min}}\left(\mathbf{C}^T\mathbf{W}\mathbf{C}\right).
\end{aligned}
$$
Unfortunately, the two objective functions provide a little different $\mathbf{p}$ with simialar objectives (after transformation).

In [11]:
## -------------------- 1st E-opt -------------------- ##
problem = maximize(eigmin(V⁻¹))
problem.constraints += p .>= 0
problem.constraints += sum(p) == 1
problem

const MOI = MathOptInterface
solver = Hypatia.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("verbose"), 0)
@time solve!(problem, solver)

# Check the status, optimal value
E_opt1_obj = round(problem.optval, digits = 3)
E_opt1_p = round.(p.value, digits = 3)
println("Obj = ", E_opt1_obj)
vcat(E_opt1_p, [E_opt1_obj])

  0.396039 seconds (537.45 k allocations: 35.442 MiB, 97.56% compilation time)
Obj = 0.077


12×1 Matrix{Float64}:
 0.266
 0.158
 0.114
 0.055
 0.042
 0.057
 0.055
 0.042
 0.057
 0.08
 0.074
 0.077

Note that $1/0.077 = 12.99 \approx 13$, which is `obj` Hua obtained. I will show this transformed `Obj` in the table.

In [12]:
sum(p.value) ≈ 1.0

true

In [13]:
E_opt1_n = proj_integer_simplex(100vec(p.value), 100)
display(E_opt1_n)
E_opt1_obj_n = eigmin((transpose(C) * Diagonal(E_opt1_n / 100) * C))
E_opt1_obj_n = round(E_opt1_obj_n, digits = 3)

11-element Vector{Float64}:
 27.0
 16.0
 11.0
  5.0
  4.0
  6.0
  6.0
  4.0
  6.0
  8.0
  7.0

0.077

In [14]:
sum(E_opt1_n) == 100

true

In [15]:
## -------------------- 2nd E-opt -------------------- ##
problem = minimize(1 / eigmin(V⁻¹))
# problem.constraints += p .>= 0
problem.constraints += sum(p) == 1
problem

const MOI = MathOptInterface
solver = Hypatia.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("verbose"), 0)
@time solve!(problem, solver)

# Check the status, optimal value
E_opt2_obj = round(problem.optval, digits = 3)
E_opt2_p = round.(p.value, digits = 3)
println("Obj = ", E_opt2_obj)
vcat(E_opt2_p, [E_opt2_obj])

  1.701458 seconds (3.34 M allocations: 227.574 MiB, 3.73% gc time, 99.45% compilation time)
Obj = 13.0


12×1 Matrix{Float64}:
  0.258
  0.162
  0.118
  0.054
  0.041
  0.059
  0.054
  0.041
  0.059
  0.074
  0.08
 13.0

In [16]:
# project p to n
E_opt2_n = proj_integer_simplex(100vec(p.value), 100)
display(E_opt2_n)
E_opt2_obj_n = 1 / eigmin(transpose(C) * Diagonal(E_opt2_n / 100) * C)
E_opt2_obj_n = round(E_opt2_obj_n, digits = 3)

11-element Vector{Float64}:
 26.0
 16.0
 12.0
  5.0
  4.0
  6.0
  5.0
  4.0
  6.0
  8.0
  8.0

13.021

In [17]:
sum(E_opt2_n) == 100

true

**$A$-optimal design**:

Use `trace_mpower(A, t, C)` $= \text{tr}(\mathbf{C}\mathbf{A}^t)$.

In [18]:
problem = minimize(trace_mpower(V⁻¹, -1, Matrix(I(6))))
problem.constraints += p .>= 0
problem.constraints += sum(p) == 1
problem.constraints += V⁻¹ in :SDP
problem

const MOI = MathOptInterface
solver = Hypatia.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("verbose"), 0)
@time solve!(problem, solver)

# Check the status, optimal value
A_opt_obj = round(problem.optval, digits = 3)
A_opt_p = round.(p.value, digits = 3)
println("Obj = ", A_opt_obj)
vcat(A_opt_p, [A_opt_obj])

  2.203660 seconds (4.15 M allocations: 277.410 MiB, 2.50% gc time, 98.36% compilation time)
Obj = 38.925


12×1 Matrix{Float64}:
  0.2
  0.101
  0.104
  0.086
  0.051
  0.068
  0.086
  0.051
  0.068
  0.106
  0.08
 38.925

In [19]:
sum(p.value) ≈ 1.0

true

In [20]:
# project p to n
A_opt_n = proj_integer_simplex(100vec(p.value), 100)
display(A_opt_n)
A_opt_obj_n = trace_mpower(C' * Diagonal(A_opt_n / 100) * C, -1, I(6))
A_opt_obj_n = round(A_opt_obj_n, digits = 3)

11-element Vector{Float64}:
 20.0
 10.0
 10.0
  9.0
  5.0
  7.0
  9.0
  5.0
  7.0
 10.0
  8.0

38.946

Let's merge the three results in table.

In [21]:
using DataFrames
treatment = [
    "A1B1", "A2B1", "A3B1",
    "A1B2", "A2B2", "A3B2",
    "A1B3", "A2B3", "A3B3",
    "A1B4", "A2B4", "Obj"
]

println("Approximate Optimal Design")
df_Q2 = DataFrame(
    design_pt = treatment,
    D_opt    = vcat(D_opt_p,  [D_opt_obj])[:],
    E_opt1   = vcat(E_opt1_p,  [1 / E_opt1_obj])[:],
    E_opt2   = vcat(E_opt2_p, [E_opt2_obj])[:],
    # E_opt3   = vcat(E_opt3_p, [- 1 / E_opt3_obj])[:],
    A_opt    = vcat(A_opt_p,  [A_opt_obj])[:],
    D_opt_n  = vcat(D_opt_n,  [D_opt_obj_n])[:],
    E_opt1_n = vcat(E_opt1_n,  [1 / E_opt1_obj_n])[:],
    E_opt2_n = vcat(E_opt2_n, [E_opt2_obj_n])[:],
    # E_opt3_n = vcat(E_opt3_n, [- 1/ E_opt3_obj_n])[:],
    A_opt_n  = vcat(A_opt_n,  [A_opt_obj_n])[:]
)

Approximate Optimal Design


Row,design_pt,D_opt,E_opt1,E_opt2,A_opt,D_opt_n,E_opt1_n,E_opt2_n,A_opt_n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,A1B1,0.082,0.266,0.258,0.2,8.0,27.0,26.0,20.0
2,A2B1,0.082,0.158,0.162,0.101,8.0,16.0,16.0,10.0
3,A3B1,0.097,0.114,0.118,0.104,10.0,11.0,12.0,10.0
4,A1B2,0.082,0.055,0.054,0.086,8.0,5.0,5.0,9.0
5,A2B2,0.082,0.042,0.041,0.051,8.0,4.0,4.0,5.0
6,A3B2,0.097,0.057,0.059,0.068,10.0,6.0,6.0,7.0
7,A1B3,0.082,0.055,0.054,0.086,8.0,6.0,5.0,9.0
8,A2B3,0.082,0.042,0.041,0.051,8.0,4.0,4.0,5.0
9,A3B3,0.097,0.057,0.059,0.068,10.0,6.0,6.0,7.0


## Q3 (30 pts) Find exact optimal designs

Using mixed-integer semidefinite programming (SDP) software to find the exact D-, E-, and A-optimal designs for this clinical trial **with $n=100$**.

Hint: This is what I got using Pajarito with Gurobi as the `oa_solver` and Mosek as the `conic_solver`.

```
Exact Optimal Design
┌───────────┬─────────┬─────────┬─────────┐
│ design_pt │   D_opt │   E_opt │   A_opt │
│    String │ Float64 │ Float64 │ Float64 │
├───────────┼─────────┼─────────┼─────────┤
│      A1B1 │   8.000 │  34.000 │  20.000 │
│      A2B1 │   8.000 │  10.000 │  10.000 │
│      A3B1 │  10.000 │  10.000 │  10.000 │
│      A1B2 │   8.000 │   9.000 │   9.000 │
│      A2B2 │   8.000 │   0.000 │   5.000 │
│      A3B2 │  10.000 │   6.000 │   7.000 │
│      A1B3 │   8.000 │   8.000 │   9.000 │
│      A2B3 │   8.000 │   4.000 │   5.000 │
│      A3B3 │  10.000 │   4.000 │   7.000 │
│      A1B4 │  11.000 │   9.000 │  10.000 │
│      A2B4 │  11.000 │   6.000 │   8.000 │
│       Obj │   8.988 │  13.011 │  38.946 │
└───────────┴─────────┴─────────┴─────────┘
```

In [11]:
using JuMP, Pajarito, Gurobi, Mosek, MosekTools

In [112]:
# Define the problem
model = Model(
    optimizer_with_attributes(
        Pajarito.Optimizer,
        MOI.Silent() => true,
        "oa_solver" => optimizer_with_attributes(
            Gurobi.Optimizer,
            MOI.Silent() => true
        ),
        "conic_solver" => optimizer_with_attributes(
            Mosek.Optimizer,
            MOI.Silent() => true
        )
    )
)
# Set a time limit for the solver
set_attribute(model, "time_limit", 60)
MOI.set(model, MOI.RawOptimizerAttribute("verbose"), 0)

**$D$-optimal design**

Use `MOI.LogDetConeSquare` to obtain a log-determinant.

In [113]:
# Define the variables
unregister(model, :n)
@variable(model, n[1:m], Int)
C = Matrix{Float64}(C)
q = size(C, 2) # 6
unregister(model, :V⁻¹)
@expression(model, V⁻¹[i=1:q, j=1:q], sum(C[k, i] * C[k, j] * n[k] for k in 1:m))
# @variable(model, V⁻¹[1:q, 1:q])
unregister(model, :obj)
@variable(model, obj)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-21


obj

In [116]:
import MathOptInterface as MOI
# Define the constraints
# @constraint(model, V⁻¹ .== [sum(C[k, i] * C[k, j] * n[k] for k in 1:m) for i in 1:q, j in 1:q])
@constraint(model, V⁻¹ >= 0, PSDCone())
@constraint(model, n .>= 0)
@constraint(model, sum(n) == 100)
@constraint(model, [obj; 1; vec(V⁻¹)] in MOI.LogDetConeSquare(q))

@objective(model, Max, obj)
optimize!(model)

┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/tomokiokuno/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/tomokiokuno/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/tomokiokuno/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/tomokiokuno/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/tomokiokuno/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396


In [117]:
# Check the status, optimal value
if termination_status(model) == MOI.OPTIMAL
    D_opt_n = value.(n)
    # D_opt_obj = round(objective_value(model), digits = 3)
    D_opt_obj = - logdet(transpose(C) * Diagonal(D_opt_n) * C / 100)
    println("Obj = ", D_opt_obj)
    println("n = ", D_opt_n)
else
    println("The problem did not solve to optimality.")
end
# check constraint satisfication
sum(D_opt_n) == 100

Obj = 8.988939362005667
n = [8.0, 9.0, 9.0, 8.0, 8.0, 10.0, 8.0, 8.0, 10.0, 11.0, 11.0]


true

**$E$-optimal design**

In [131]:
# Define the variables
unregister(model, :n)
@variable(model, n[1:m], Int)
C = Matrix{Float64}(C)
q = size(C, 2) # 6
unregister(model, :V⁻¹)
# @expression(model, V⁻¹[i=1:q, j=1:q], sum(C[k, i] * C[k, j] * n[k] for k in 1:m))
@variable(model, V⁻¹[1:q, 1:q])
unregister(model, :obj)
@variable(model, obj)
unregister(model, :Z)
@variable(model, Z[1:q, 1:q])

6×6 Matrix{VariableRef}:
 Z[1,1]  Z[1,2]  Z[1,3]  Z[1,4]  Z[1,5]  Z[1,6]
 Z[2,1]  Z[2,2]  Z[2,3]  Z[2,4]  Z[2,5]  Z[2,6]
 Z[3,1]  Z[3,2]  Z[3,3]  Z[3,4]  Z[3,5]  Z[3,6]
 Z[4,1]  Z[4,2]  Z[4,3]  Z[4,4]  Z[4,5]  Z[4,6]
 Z[5,1]  Z[5,2]  Z[5,3]  Z[5,4]  Z[5,5]  Z[5,6]
 Z[6,1]  Z[6,2]  Z[6,3]  Z[6,4]  Z[6,5]  Z[6,6]

In [135]:
# Define the constraints
# @constraint(model, V⁻¹ >= 0, PSDCone())
@constraint(model, V⁻¹ .== [sum(C[k, i] * C[k, j] * n[k] for k in 1:m) for i in 1:q, j in 1:q])
@constraint(model, sum(n) == 100)
# Z is lower triangular
@constraints model begin
    [i in 1:q, j in (i+1):q], Z[i, j] == 0
end

Variable(model, Mat, [V⁻¹ Z; Z' Diagonal(Z)])
@constraint(model, Mat >= 0, PSDCone())

MethodError: MethodError: Cannot `convert` an object of type AffExpr to an object of type VariableRef

Closest candidates are:
  convert(::Type{T}, !Matched::T) where T
   @ Base Base.jl:84
  (::Type{GenericVariableRef{T}} where T)(::Any, !Matched::Any)
   @ JuMP ~/.julia/packages/JuMP/Gwn88/src/variables.jl:258


In [121]:
# Define the objective is sum of log diagonal Z
@objective(model, Max, sum(log(Z[i, i]) for i in 1:q))

# Set a time limit for the solver
set_attribute(model, "time_limit", 60)

# Solve the problem
@time optimize!(model)

# Check the status, optimal value
if termination_status(model) == MOI.OPTIMAL
    A_opt_obj = round(objective_value(model), digits = 3)
    A_opt_n = round.(value.(n), digits = 3)
    println("Obj = ", A_opt_obj)
    println("n = ", A_opt_n)
else
    println("The problem did not solve to optimality.")
end

ErrorException: The solver does not support an objective function of type MathOptInterface.ScalarNonlinearFunction.

## Q4 (30 bonus points) Optimal design with nuisance parameters

Suppose the regression coefficients of linear model $\boldsymbol{\beta}$ is partitioned as $\boldsymbol{\beta} = (\boldsymbol{\beta}_0^T, \boldsymbol{\beta}_1^T)^T$, where $\boldsymbol{\beta}_0$ are nuisance parameters and $\boldsymbol{\beta}_1$ are parameters of primary interest. Given an approximate design $\mathbf{p} = (p_1, \ldots, p_m)$, let the information matrix be partitioned accordingly
$$
\mathbf{I}(\mathbf{p}) = \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T =  \begin{pmatrix}
\mathbf{I}_{00} & \mathbf{I}_{01} \\
\mathbf{I}_{10} & \mathbf{I}_{11}
\end{pmatrix}.
$$
Then the information matrix for $\boldsymbol{\beta}_1$ adjusted for nuisance parameter $\boldsymbol{\beta}_0$ is
$$
\mathbf{I}_{1 \mid 0}(\mathbf{p}) = \mathbf{I}_{11} - \mathbf{I}_{10} \mathbf{I}_{00}^{-1} \mathbf{I}_{01}.
$$

Revisiting the 3x4 factorial design problem in Q1, suppose the drug company only cares about the estimation of A treatment effects. Find the approximate D-, E-, and A-optimal designs.

**Solution**

In [337]:
C = Matrix{Float64}(C)
V⁻¹ = transpose(C) * Diagonal(p) * C
I00 = V⁻¹[[1,4,5,6], [1,4,5,6]]
I01 = V⁻¹[[1,4,5,6], [2,3]]
I11 = V⁻¹[[2,3], [2,3]]
E = Matrix{Float64}(I, 2, 2)
X = Semidefinite(2)
Mat = [I00 I01 zeros(4, 2);
       transpose(I01) I11 E;
       zeros(2, 4) E X]

transpose (affine; real)
└─ hcat (affine; real)
   ├─ transpose (affine; positive)
   │  └─ hcat (affine; positive)
   │     ├─ …
   │     ├─ …
   │     └─ …
   ├─ transpose (affine; positive)
   │  └─ hcat (affine; positive)
   │     ├─ …
   │     ├─ …
   │     └─ …
   └─ transpose (affine; real)
      └─ hcat (affine; real)
         ├─ …
         ├─ …
         └─ …

In [355]:
problem = minimize(-logdet(X))

problem.constraints += p .>= 0 # not needed using Positive()
problem.constraints += sum(p) == 1
problem.constraints += Mat in :SDP
problem

const MOI = MathOptInterface
solver = Hypatia.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("verbose"), 0)

@time solve!(problem, solver)

# Check optimal value and optimal solution
D_opt_obj = round(problem.optval, digits = 3)
D_opt_p = round.(p.value, digits = 3)
println("Obj = ", D_opt_obj)
vcat(D_opt_p, [D_opt_obj])

  0.033309 seconds (97.50 k allocations: 4.584 MiB)
Obj = -32.021


┌ Warning: Problem status ALMOST_OPTIMAL; solution may be inaccurate.
└ @ Convex /Users/tomokiokuno/.julia/packages/Convex/b2S4H/src/solution.jl:342


12×1 Matrix{Float64}:
   0.09
   0.09
   0.091
   0.09
   0.09
   0.091
   0.09
   0.09
   0.091
   0.094
   0.094
 -32.021

In [350]:
problem = minimize(eigmax(X))

problem.constraints += p .>= 0 # not needed using Positive()
problem.constraints += sum(p) == 1
problem.constraints += Mat in :SDP
problem

const MOI = MathOptInterface
solver = Hypatia.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("verbose"), 0)

@time solve!(problem, solver)

# Check optimal value and optimal solution
E_opt_obj = round(problem.optval, digits = 3)
E_opt_p = round.(p.value, digits = 3)
println("Obj = ", E_opt_obj)
vcat(E_opt_p, [E_opt_obj])

  0.019972 seconds (27.46 k allocations: 1.686 MiB)
Obj = 8.0


12×1 Matrix{Float64}:
 0.141
 0.057
 0.084
 0.14
 0.057
 0.083
 0.14
 0.057
 0.083
 0.08
 0.08
 8.0

In [351]:
problem = minimize(tr(X))

problem.constraints += p .>= 0 # not needed using Positive()
problem.constraints += sum(p) == 1
problem.constraints += Mat in :SDP

problem

const MOI = MathOptInterface
solver = Hypatia.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("verbose"), 0)

@time solve!(problem, solver)

# Check optimal value and optimal solution
A_opt_obj = round(problem.optval, digits = 3)
A_opt_p = round.(p.value, digits = 3)
println("Obj = ", A_opt_obj)
vcat(A_opt_p, [A_opt_obj])

MethodError: MethodError: no method matching +(::Vector{Constraint}, ::Nothing)

Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:587
  +(!Matched::MutableArithmetics.Zero, ::Any)
   @ MutableArithmetics ~/.julia/packages/MutableArithmetics/iovKe/src/rewrite.jl:64
  +(::Any, !Matched::MutableArithmetics.Zero)
   @ MutableArithmetics ~/.julia/packages/MutableArithmetics/iovKe/src/rewrite.jl:65
  ...


In [347]:
using DataFrames
treatment = [
    "A1B1", "A2B1", "A3B1",
    "A1B2", "A2B2", "A3B2",
    "A1B3", "A2B3", "A3B3",
    "A1B4", "A2B4", "Obj"
]

df_Q2 = DataFrame(
    design_pt = treatment,
    D_opt   = vcat(D_opt_p,  [D_opt_obj])[:],
    E_opt   = vcat(E_opt_p,  [E_opt_obj])[:],
    A_opt   = vcat(A_opt_p,  [A_opt_obj])[:],
)

Row,design_pt,D_opt,E_opt,A_opt
,String,Float64,Float64,Float64
1,A1B1,0.09,0.141,0.139
2,A2B1,0.09,0.057,0.098
3,A3B1,0.091,0.084,0.098
4,A1B2,0.09,0.14,0.138
5,A2B2,0.09,0.057,0.097
6,A3B2,0.091,0.083,0.097
7,A1B3,0.09,0.14,0.138
8,A2B3,0.09,0.057,0.097
9,A3B3,0.091,0.083,0.097
